In [13]:
import os
import pandas as pd

In [14]:
Crops = ["quantitySold", 
         "valueSales", 
         "cropProduction", 
         "irrigatedArea", 
         "cultivatedArea", 
         "organicProductionType", 
         "variableCostsCrops", 
         "landValue", 
         "quantityUsed", 
         "sellingPrice"]

Animals = ["numberOfAnimals", 
           "numberOfAnimalsSold", 
           "valueSoldAnimals", 
           "numberAnimalsRearingBreading", 
           "valueAnimalsRearingBreading", 
           "numberAnimalsForSlaughtering", 
           "valueSlaughteredAnimals", 
           "milkTotalProduction", 
           "milkProductionSold", 
           "milkTotalSales", 
           "milkVariableCosts", 
           "dairyCows", 
           "variableCostsAnimals", 
           "woolTotalProduction", 
           "woolProductionSold", 
           "eggsTotalProduction", 
           "eggsProductionSold", 
           "eggsTotalSales", 
           "manureTotalSales", ]
    
Subsidies = [
    "policyIdentifier", 
    "policyDescription", 
    "isCoupled", 
    "value",
]

Farm = [
    "lat", 
    "long", 
    "altitude", 
    "farmCode", 
    "technicalEconomicOrientation", 
    "weight_ra", 
    "regionLevel1Name", 
    "regionLevel2Name", 
    "regionLevel3Name", 
    "regionLevel1", 
    "regionLevel2", 
    "regionLevel3", 
    "weight_reg", ]

Economic = [
    "agriculturalLandArea",
    "agriculturalLandValue",
    "agriculturalLandHectaresAdquisition",
    "landImprovements", 
    "forestLandArea",
    "forestLandValue",
    "farmBuildingsValue", 
    "machineryAndEquipment", 
    "intangibleAssetsTradable", 
    "intangibleAssetsNonTradable", 
    "otherNonCurrentAssets", 
    "longAndMediumTermLoans", 
    "totalCurrentAssets", 
    "farmNetIncome", 
    "grossFarmIncome", 
    "subsidiesOnInvestments", 
    "vatBalanceOnInvestments", 
    "totalOutputCropsAndCropProduction", 
    "totalOutputLivestockAndLivestockProduction", 
    "otherOutputs", 
    "totalIntermediateConsumption", 
    "taxes", 
    "vatBalanceExcludingInvestments", 
    "fixedAssets", 
    "depreciation", 
    "totalExternalFactors", 
    "machinery",
    "rentBalance", ]



Holder = [
    "holderAge", 
    "holderGender", 
    "holderSuccessors", 
    "holderSuccessorsAge", 
    "holderFamilyMembers", 
    "yearNumber", ]

# Set forbidden directions
Variable defined for ABM can be goruped in six main groups namely:  
* ***Farm***: geospatial and techno-economical orientation of the farm. 
* ***Holder***: features of the holder of the farm. 
* ***Crops***: data about the crops produced, their areas, quantities...
* ***Animals***: data about the animalas available in the farm, their producions and economical revenues.
* ***Subsidies***: information about the subsidies managed in the farm. 
* ***Economic***: economical indicators of the farm at the end of the accountancy exercise.  
  
Considerig these major groups, some assumptions will be made to set the forbidden directions:
1. Nothing affects ***Farm***. This will ensure that the features of the original population are replicated in the synthetic population.  
2. ***Farm*** will affect all other groups. According to geospatial attributes, other values are generated.  
3. ***Holder*** is only affected by ***Farm***.
4. ***Crops*** is only affected by ***Farm***.
5. ***Animals*** is only affected by ***Farm***.
6. ***Subsidies*** is affected by ***Farm***, ***Holder***, ***Crops*** and ***Animals***, but it is not affected by ***Economic***.
7. ***Economic*** is affected by all other groups, ***Farm***, ***Holder***, ***Crops***, ***Animals*** and ***Subsidies***.  
  
In this sense, and to define the forbidden directions between gorups:  
1. ***Farm*** is not affected by ***Holder***, ***Crops***, ***Animals***, ***Subsidies*** and ***Economic***.  
2. ***Holder*** is not affected by ***Crops***, ***Animals***, ***Subsidies*** and ***Economic***.  
4. ***Crops*** is not affected by ***Holder***, ***Animals***, ***Subsidies*** and ***Economic***.  
5. ***Animals*** is not affected by ***Holder***, ***Crops***, ***Subsidies*** and ***Economic***.  
6. ***Subsidies*** is not affected by ***Economic***.  
  

## Inter-groups forbidden directions

In [15]:
fd = pd.DataFrame(columns=["from", "to"])

In [16]:
fd_groups = {
    "Farm": ["Holder", "Crops", "Animals", "Subsidies", "Economic"], 
    "Holder": ["Crops", "Animals", "Subsidies", "Economic"], 
    "Crops": ["Holder", "Animals", "Subsidies", "Economic"], 
    "Animals": ["Holder", "Crops", "Subsidies", "Economic"], 
    "Subsidies": ["Economic"], 
    }


In [17]:
for to_group in fd_groups.keys():
    for to_var in eval(to_group):
        for from_group in fd_groups[to_group]:
            for from_var in eval(from_group):
                fd = pd.concat([fd, pd.DataFrame({"from": [from_var], "to": [to_var]})])


# Intra-groups forbidden directions

In [18]:
#   Son: Parents
intra_group_crops = {
    "cultivatedArea":        [], 
    "irrigatedArea":         ["cultivatedArea", "landValue", ], 
    "cropProduction":        ["cultivatedArea", "irrigatedArea", "organicProductionType", "variableCostsCrops", "landValue", ], 
    "quantitySold":          ["cultivatedArea", "irrigatedArea", "cropProduction", "organicProductionType", "variableCostsCrops", "landValue", ], 
    "quantityUsed":          ["cultivatedArea", "irrigatedArea", "cropProduction", "organicProductionType", "variableCostsCrops", "landValue", ], 
    "valueSales":            ["cultivatedArea", "irrigatedArea", "cropProduction", "quantitySold", "organicProductionType", "variableCostsCrops", "landValue", "sellingPrice"], 
    "organicProductionType": ["cropProduction", "quantitySold", "valueSales", ], 
    "variableCostsCrops":    ["cultivatedArea", "irrigatedArea", "organicProductionType", "landValue", ], 
    "landValue":             ["cultivatedArea", "irrigatedArea", ], 
    "sellingPrice":          ["organicProductionType"], 
}

inter_group_animals = {
    'numberOfAnimals': [],
    'numberOfAnimalsSold':          ["numberOfAnimals", ],
    'valueSoldAnimals':             ["numberOfAnimals", "numberOfAnimalsSold", ],
    'numberAnimalsRearingBreading': ["numberOfAnimals", ],
    'valueAnimalsRearingBreading':  ["numberOfAnimals", "numberAnimalsRearingBreading", ],
    'numberAnimalsForSlaughtering': ["numberOfAnimals", ],
    'valueSlaughteredAnimals':      ["numberOfAnimals", "numberAnimalsForSlaughtering", ],
    'milkTotalProduction':          ["numberOfAnimals", "dairyCows", ],
    'milkProductionSold':           ["numberOfAnimals", "milkTotalProduction", "dairyCows", ], 
    'milkTotalSales':               ["numberOfAnimals", "milkTotalProduction", "milkProductionSold", "dairyCows", ],
    'milkVariableCosts':            ["numberOfAnimals", "numberOfAnimalsSold", "valueSoldAnimals", "numberAnimalsRearingBreading", "valueAnimalsRearingBreading", "numberAnimalsForSlaughtering", "valueSlaughteredAnimals", "milkTotalProduction", "milkProductionSold", "milkTotalSales", "milkVariableCosts", "dairyCows", "variableCostsAnimals", "woolTotalProduction", "woolProductionSold", "eggsTotalProduction", "eggsProductionSold", "eggsTotalSales", "manureTotalSales", ],
    'dairyCows':                    ["numberOfAnimals", ],
    'variableCostsAnimals':         ["numberOfAnimals", "numberAnimalsRearingBreading", "numberAnimalsForSlaughtering", "milkTotalProduction", "milkVariableCosts", "dairyCows", "woolTotalProduction", "eggsTotalProduction", ],
    'woolTotalProduction':          ["numberOfAnimals", ],
    'woolProductionSold':           ["numberOfAnimals", "woolTotalProduction", ],
    'eggsTotalProduction':          ["numberOfAnimals", "eggsProductionSold", "eggsTotalSales", "manureTotalSales", ],
    'eggsProductionSold':           ["numberOfAnimals", "eggsTotalProduction", ],
    'eggsTotalSales':               ["numberOfAnimals", "eggsTotalProduction", "eggsProductionSold", ],
    'manureTotalSales':             ["numberOfAnimals", ],
    #'valueSold':                    ["valueSoldAnimals", "valueAnimalsRearingBreading", "valueSlaughteredAnimals", "milkProductionSold", "milkTotalSales", "woolProductionSold", "eggsProductionSold", "eggsTotalSales", "manureTotalSales", "valueSold", ]
    }


In [ ]:
for from_var in intra_group_crops.keys():
    for to_var in intra_group_crops[from_var]:
        print(f'from {from_var} to {to_var}')
        fd = pd.concat([fd, pd.DataFrame({"from": [from_var], "to": [to_var]})])

In [ ]:
for from_var in inter_group_animals.keys():
    for to_var in inter_group_animals[from_var]:
        print(f'from {from_var} to {to_var}')
        fd = pd.concat([fd, pd.DataFrame({"from": [from_var], "to": [to_var]})])

In [ ]:
fd

# Subsidies

In [22]:
for from_var in ["policyDescription", "policyIdentifier", "isCoupled"]:    
    for to_group in fd_groups.keys():
        for to_var in eval(to_group):
            fd = pd.concat([fd, pd.DataFrame({"from": [from_var], "to": [to_var]})])
            #print(f"from: {[from_var]}, to: {[to_var]})]")
            
for from_group in fd_groups[to_group]:
    for from_var in eval(from_group):
        for to_var in ["policyDescription", "policyIdentifier", "isCoupled"]:    
            fd = pd.concat([fd, pd.DataFrame({"from": [from_var], "to": [to_var]})])
            #print(f"from: {[from_var]}, to: {[to_var]})]")


In [ ]:
# Categorical policy variables do not affect anything, and they are not affected by anyth
fd

In [24]:
fd.to_csv("./../data/use_case_andalusia/metadata/forbidden_directions_v2.csv", index=False)
fd.to_csv("./../data/use_case_andalusia2/metadata/forbidden_directions_v2.csv", index=False)
fd.to_csv("./../data/use_case_andalusia3/metadata/forbidden_directions_v2.csv", index=False)
fd.to_csv("./../data/use_case_andalusia4/metadata/forbidden_directions_v2.csv", index=False)
